In [1]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from dotenv import load_dotenv
import os
from pydantic import Field
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen
from typing import Optional
from spire.doc import Document,FileFormat
from spire.doc.common import *
import requests

In [2]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')
pse=os.getenv('pse')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:
    deep_search_results:list[str]
    quick_search_results:list[str]
    edited_deep_search_result:str
    presentation_slides:dict



async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results.append(res)
    return str(res)

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return str(res.data)


def google_image_search(query:str):
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": google_api_key,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  

@dataclass
class Research_edits:
    edits:str = Field(description='the edited reasearch')
editor_agent=Agent(llm,tools=[google_image_search],result_type=Research_edits, system_prompt="you are an editor, you are given a query, a deep search result, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions")
async def research_editor_tool(ctx: RunContext[Deps], query:str):
    """
    This function is used to edit the deep search result to make it more accurate following the query's instructions.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    res=await editor_agent.run(f'query:{query}, deep_search_results:{ctx.deps.deep_search_results}, quick_search_results:{ctx.deps.quick_search_results}')
    ctx.deps.edited_deep_search_result=res.data.edits
    return str(res.data.edits)


async def presentation_agent(
    ctx: RunContext[Deps],
    style: Optional[str] = "professional",
    instruction: Optional[str] = 'None'
) -> dict:
    """
    Generate a presentation based on the deep search result.

    Args:
        style: Presentation style (default: "professional")
        instruction: Additional instructions for presentation generation (default: None)

    Returns:
        dict: The presentation slides
    """
    search_content = (ctx.deps.edited_deep_search_result 
                     if ctx.deps.edited_deep_search_result 
                     else ctx.deps.deep_search_results[-1])
    
    res = await presentation.chat(search_content, presentation_style=style, instruction=instruction)
    ctx.deps.presentation_slides = res
    return 'presentation generated'

@dataclass
class page_edit:
    edits: str = Field(description='the edited page in html format')

page_editor=Agent(llm, result_type=page_edit, system_prompt='you are a page editor, edit the mentionned page(s) based on the instructions.')
async def page_editor_tool(ctx: RunContext[Deps], page_number: int, instructions: str):
    """
    This function is used to edit the mentionned page(s) based on the instructions.
    Args:
        page_number (int): The number of the page to edit
        instructions (str): The instructions for the page edits
    Returns:
        str: The edited page
    """
    result=await page_editor.run(f'page:{ctx.deps.presentation_slides[f"page_{page_number}"]}, instructions:{instructions}')
    ctx.deps.presentation_slides[f"page_{page_number}"]=result.data.edits
    return f'page{page_number} edited'


In [3]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

In [4]:
class Main_agent:
    def __init__(self):
        self.agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, leverage what tool(s) to use but do not use the presentation agent unless the user asks for it,\
                          always show the output of the tools, except for the presentation agent",
                          tools=[deep_research_agent,page_editor_tool,presentation_agent,research_editor_tool,quick_research_agent])
        self.deps=Deps( deep_search_results=[], quick_search_results=[],edited_deep_search_result="", presentation_slides={})
        self.memory=Message_state(messages=[])

    async def chat(self, query:str):
        result = await self.agent.run(query,deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data
    
    def reset(self):
        self.memory.messages=[]
        self.deps=Deps( deep_search_results=[], quick_search_results=[],edited_deep_search_result="", presentation_slides={})



In [5]:
agent=Main_agent()

In [6]:
await agent.chat('do a deep research on toast, the pos system for restaurants, all its features and costs, include a table with the features and costs')

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\deep_research.py:177: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  response=await self.graph.run(preliminary_search_node(),state=self.state)
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\.venv\Lib\site-packages\pydantic\main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `str` with value `'https://www.reddit.com/r...of-Sale-POS-Renewal-FAQ'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


'Okay, I have performed a deep research on Toast POS system for restaurants, including its features and costs, and generated a table summarizing the information. The output is above.\n'

In [7]:
md='# Blind Snakes of the Bahamas\n\nBlind snakes, belonging to the family Typhlopidae, are small, burrowing snakes with a worm-like appearance. Several species are found in the Bahamas, including the Bahamian slender blind snake (Cubatyphlops biminiensis) and the Brown Blind Snake (Typhlops lumbricalis).\n\n## Species\n\n*   **Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n*   **Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.\n\n## Characteristics\n\nBlind snakes are typically small and slender, resembling worms. They are adapted for burrowing, with smooth scales and reduced eyes.\n\n## Habitat and Distribution\n\nBahamian blind snakes inhabit various environments, including coppice dry forest habitat. They are often found under rocks, in rotting logs, and in the soil.\n\n## Conservation\n\nWhile some blind snake species are listed as Least Concern, they still face threats such as habitat loss and degradation due to human activities like urbanization and agriculture. Conservation efforts are important to ensure the survival of these unique reptiles.\n\n##Interesting facts\n\nIt is really hard to see these SMALL snakes in the Bahamas. Take one look at them, and you will notice they look more like small worms than the other snakes that live in the Bahamas.\n\n## References\n\n*   Animalia.bio: [https://animalia.bio/bahamian-slender-blind-snake](https://animalia.bio/bahamian-slender-blind-snake)\n*   Wikipedia: [https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake](https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake)\n*   Hummingbirdsplus.org: [https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/](https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/)\n*   Biminisharklab.com: [https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake](https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake)\n*   journals.ku.edu: [https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618](https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618)\n*   Allenpress.com: [https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with](https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with)\n*   Birdwatchinghq.com: [https://birdwatchinghq.com/snakes-of-the-Bahamas/](https://birdwatchinghq.com/snakes-of-the-Bahamas/)\n*   Photoguides.org: [https://photoguides.org/snakes-in-the-bahamas/](https://photoguides.org/snakes-in-the-bahamas/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/](https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/](https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/)\n\n'


In [10]:
agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are a research assistant, you are given a query, leverage what tool(s) to use but do not use the presentation agent unless the user asks for it,                          always show the output of the tools, except for the presentation agent', timestamp=datetime.datetime(2025, 4, 3, 19, 46, 52, 882895, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='do a deep research on toast, the pos system for restaurants, all its features and costs, include a table with the features and costs', timestamp=datetime.datetime(2025, 4, 3, 19, 46, 52, 882902, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='deep_research_agent', args={'query': 'toast pos system for restaurants features and costs, include a table with the features and costs'}, tool_call_id='pyd_ai_e20a5dfbae1f4684ad98fe721eedd871', part_kind='tool-call')], model_name

In [7]:
agent.deps.deep_search_results[0]

{'title': 'Toast POS System for Restaurants: Features and Costs',
 'image_url': 'https://images.ctfassets.net/tgv5pu8pgmos/2fd6V5tmebdFzAzDTF9II8/c19a42bdb7c9029efd3ce62d232388a7/img-toast-pos-redesign-hero-image-sm.png',
 'paragraphs': [{'title': 'Introduction',
   'content': 'Toast POS is a comprehensive, cloud-based restaurant management system tailored for the food service industry, offering a range of features like menu management, order tracking, payment processing, and reporting. Its pricing structure includes subscription-based software plans, hardware options, and payment processing fees, with options for pay-as-you-go or upfront hardware costs affecting processing rates. While Toast offers robust features and specialized tools, particularly for larger or multi-location businesses, its initial hardware and software expenses, along with potential contract complexities and cancellation fees, can be a barrier for some smaller establishments. However, its commitment to customer su

In [10]:
deepsearch.state.research_results.references

'https://www.reddit.com/r/ToastPOS/comments/17dfmoe/cancelling_toast_before_install_day/, https://pos.toasttab.com/end-user-license-agreement, https://pos.toasttab.com/terms-of-service, https://lavu.com/how-to-cancel-toast-contract/, https://central.toasttab.com/s/article/Toast-Point-of-Sale-POS-Renewal-FAQ'

In [13]:
def paper_to_markdown(paper:dict) -> str:
    """
    Convert a Paper_layout object into a markdown string.
    
    Args:
        paper (Paper_layout): The paper layout to convert
        
    Returns:
        str: The complete markdown document
    """
    markdown = []
    
    # Add title
    markdown.append(paper.get('title'))
    markdown.append("\n\n")
    if paper.get('image_url'):
        markdown.append(f"![Image]({paper.get('image_url')})\n")
        markdown.append("\n\n")
    
    # Add body sections
    for section in paper.get('paragraphs'):
        markdown.append(f"## {section.get('title')}\n")
        if section.get('content'):
            markdown.append(section.get('content'))
            markdown.append("\n")


        markdown.append("\n\n")
    
    if paper.get('table'):
        markdown.append(paper.get('table'))
        markdown.append("\n")
    # Add references
    markdown.append("## References\n")
    markdown.append(str(paper.get('references')))
    
    return "".join(markdown)

In [14]:
paper=paper_to_markdown(agent.deps.deep_search_results[0])

In [15]:
print(paper)

Toast POS System for Restaurants: Features and Costs

![Image](https://images.ctfassets.net/tgv5pu8pgmos/2fd6V5tmebdFzAzDTF9II8/c19a42bdb7c9029efd3ce62d232388a7/img-toast-pos-redesign-hero-image-sm.png)


## Introduction
Toast POS is a comprehensive, cloud-based restaurant management system tailored for the food service industry, offering a range of features like menu management, order tracking, payment processing, and reporting. Its pricing structure includes subscription-based software plans, hardware options, and payment processing fees, with options for pay-as-you-go or upfront hardware costs affecting processing rates. While Toast offers robust features and specialized tools, particularly for larger or multi-location businesses, its initial hardware and software expenses, along with potential contract complexities and cancellation fees, can be a barrier for some smaller establishments. However, its commitment to customer support, continuous updates, and integrations, solidify its 

In [16]:
import tempfile
import os

# Create a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as temp_file:
    temp_file.write(paper)
    temp_file_path = temp_file.name

# Create a new document and load from the temporary files
document = Document()
document.LoadFromFile(temp_file_path)

# Save as DOCX
document.SaveToFile("output.docx", FileFormat.Docx)

# Clean up the temporary file
os.unlink(temp_file_path)